## **Dependencies**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## **Splitting Cam Images (928 x 1280) into 2 Total Images: 1 Multispectral Image (928 x 640) and 1 RGB Image (928 x 640)**

In [ ]:
def process_rgb_images(input_folder, output_folder_rgb):
    # List all camera folders
    cam_folders = [f'cam{i}' for i in range(1, 9)]

    for cam in cam_folders:
        cam_path = os.path.join(input_folder, cam)
        output_cam_rgb_path = os.path.join(output_folder_rgb, f'{cam}_rgb')
        
        os.makedirs(output_cam_rgb_path, exist_ok=True)
        
        # List all image files in the current camera folder
        for filename in os.listdir(cam_path):
            if filename.endswith('.png'):
                image_path = os.path.join(cam_path, filename)
                
                # Open the image
                img = Image.open(image_path)
                
                # Check the image size to ensure it is 1280 x 928
                if img.size != (1280, 928):
                    print(f"Skipping {image_path}: unexpected image size {img.size}")
                    continue
                
                # Split the image to get the right half
                right_half = img.crop((640, 0, 1280, 928))
                
                # Save the right half
                right_half.save(os.path.join(output_cam_rgb_path, f'{os.path.splitext(filename)[0]}{os.path.splitext(filename)[1]}'))
                
                print(f"Processed right side of {filename}")

def process_nir_images(input_folder, output_folder_ir):
    # List all camera folders
    cam_folders = [f'cam{i}' for i in range(1, 9)]

    for cam in cam_folders:
        cam_path = os.path.join(input_folder, cam)
        output_cam_ir_path = os.path.join(output_folder_ir, f'{cam}_nir')
        
        os.makedirs(output_cam_ir_path, exist_ok=True)
        
        # List all image files in the current camera folder
        for filename in os.listdir(cam_path):
            if filename.endswith('.png'):
                image_path = os.path.join(cam_path, filename)
                
                # Open the image
                img = Image.open(image_path)
                
                # Check the image size to ensure it is 1280 x 928
                if img.size != (1280, 928):
                    print(f"Skipping {image_path}: unexpected image size {img.size}")
                    continue
                
                # Split the image to get the left half
                left_half = img.crop((0, 0, 640, 928))
                
                # Save the left half
                left_half.save(os.path.join(output_cam_ir_path, f'{os.path.splitext(filename)[0]}{os.path.splitext(filename)[1]}'))
                
                print(f"Processed left side of {filename}")

# Input and output folders
input_folder = '../data'
output_folder_rgb = '../data/'
output_folder_ir = '../data/'

# Run the functions
# process_rgb_images(input_folder, output_folder_rgb)
process_nir_images(input_folder, output_folder_ir)

### Step 1: Do Radiometric Reflectance Correction on Reference Panels ✔️
### Step 2: Labelling - aka Find ROIs (Region of Interests)
### Step ???: Use AlexNet to Segment Wheat Plot Fields
### Step 3: Compute VI (Vegetation Indices)
### Step 4: Extract Data -> Summaries

## **Calculating Radiometric Reflectance on Reference Panels**

find the average value of RGB, NIR of gray panel for each image...
Then, divide each pixek in R band by average value of gray panel in red band.  repeat this for 6 bands. This becom4es reflectance image.
Register NRI images to RGB images.  You could even use the gray panel is registration points...
Calculate NDVI at each pixel.
NDFVI = (NRI-Red)/(NRI+red).  But you have 3 NIR bands.  Which one should be used?
Either segment whole wheat field from reflectance image or NDVI image.
Then, use logic to divide the wheat field into plots.
Find ststaitsics of NDVI from each plot

### Reordering CSV files in the `model_output` directory

In [ ]:
import os
import pandas as pd

# Define the naming conventions and range
prefixes = [f'cam{i}' for i in range(1, 9)]
suffixes = ['_nir.csv', '_rgb.csv']

# Get list of CSV files in the directory
all_files = os.listdir('../model_output/')
csv_files = [f for f in all_files if any(f.startswith(prefix) and f.endswith(suffix) for prefix in prefixes for suffix in suffixes)]

# Process each CSV file
for csv_file in csv_files:
    # Read the CSV file
    file_path = f'../model_output/{csv_file}'
    df = pd.read_csv(file_path)
    
    # Remove rows with NaN in Center_X or Center_Y
    df = df.dropna(subset=['Center_X', 'Center_Y'])
    
    # Sort by Filename
    df = df.sort_values(by='Filename')
    
    # Save the cleaned and sorted dataframe back to the CSV file
    df.to_csv(file_path, index=False)
    
    print(f"Processed {csv_file}")

### Saving Plots in the `assets` directory

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Go through each camera and plot the Center_X and Center_Y values
output_folder = '../assets'

for i in range(1, 9):
    # Plot for cam{i}_nir.csv
    nir_df = pd.read_csv(f'../model_output/cam{i}_nir.csv')
    
    plt.figure(figsize=(10, 7))
    plt.scatter(nir_df['Center_X'], nir_df['Center_Y'], c='r', marker='o')
    plt.title(f'Camera {i} NIR Center_X and Center_Y values')
    plt.xlabel('Center_X')
    plt.ylabel('Center_Y')
    plt.grid(True)
    plt.savefig(f'{output_folder}/cam{i}_nir_center_x_y.png')
    plt.close()
    
    # Plot for cam{i}_rgb.csv
    rgb_df = pd.read_csv(f'../model_output/cam{i}_rgb.csv')
    
    plt.figure(figsize=(10, 7))
    plt.scatter(rgb_df['Center_X'], rgb_df['Center_Y'], c='b', marker='o')
    plt.title(f'Camera {i} RGB Center_X and Center_Y values')
    plt.xlabel('Center_X')
    plt.ylabel('Center_Y')
    plt.grid(True)
    plt.savefig(f'{output_folder}/cam{i}_rgb_center_x_y.png')
    plt.close()

In [ ]:
import pandas as pd
import cv2
import numpy as np
import os

def calculate_panel_rgb_values(csv_file, image_folder):
    # Load the CSV file
    data = pd.read_csv(csv_file)
    
    # List to store the results
    results = []

    print(f"Processing {len(data)} rows from {csv_file}")

    for index, row in data.iterrows():
        filename = row['Filename']
        center_x = row['Center_X']
        center_y = row['Center_Y']
        
        # Construct the path to the image file
        image_path = os.path.join(image_folder, filename)
        
        # Check if the file exists
        if not os.path.exists(image_path):
            # print(f"File does not exist: {image_path}")
            continue
        
        # Load the image
        image = cv2.imread(image_path)
        
        # Check if the image was loaded successfully
        if image is None:
            # print(f"Could not load image: {image_path}")
            continue
        
        # Extract a 5x5 region around the center coordinates
        x_start = max(0, center_x - 5)
        x_end = min(image.shape[1], center_x + 5)
        y_start = max(0, center_y - 5)
        y_end = min(image.shape[0], center_y + 5)
        
        region = image[y_start:y_end, x_start:x_end]
        
        # Calculate the average RGB values
        average_color = region.mean(axis=(0, 1))
        
        # Append the result
        results.append({
            'Filename': filename,
            'Center_X': center_x,
            'Center_Y': center_y,
            'Average_R': average_color[2],
            'Average_G': average_color[1],
            'Average_B': average_color[0]
        })
    
    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Define folder paths
csv_files = [f"../model_output/cam{i}_centers.csv" for i in range(1, 9)]
image_folders = [f"../model_output/cam{i}" for i in range(1, 9)]

# Iterate over the CSV files and calculate the average RGB values
for csv_file, image_folder in zip(csv_files, image_folders):
    print(f"Processing {csv_file} with images from {image_folder}")
    result_df = calculate_panel_rgb_values(csv_file, image_folder)
    print(f"Processed {len(result_df)} images for {csv_file}")
    print(result_df)

### Performing Radiometric Reflectance Corrections

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def vi(img, nir_x_center, nir_y_center, rgb_x_center, rgb_y_center):
    # Split the image into NIR and RGB
    nir = img[:, :640, :].astype(np.float64)
    rgb = img[:, 641:1280, :].astype(np.float64)
    
    # Extract a 5x5 region around the RGB center coordinates
    rgb_gray = img[rgb_y_center - 5:rgb_y_center + 5, rgb_x_center - 5:rgb_x_center + 5, :]
    
    # Extract the red, green, and blue bands from the gray region
    rgb_gray_red, rgb_gray_green, rgb_gray_blue = rgb_gray[:, :, 2], rgb_gray[:, :, 1], rgb_gray[:, :, 0]
    rgb_gray_red_avg, rgb_gray_green_avg, rgb_gray_blue_avg = np.mean(rgb_gray_red), np.mean(rgb_gray_green), np.mean(rgb_gray_blue)
    
    # Convert the RGB image to red, green, blue bands
    red, green, blue = rgb[:, :, 2], rgb[:, :, 1], rgb[:, :, 0]
    rgb_red_ref, rgb_green_ref, rgb_blue_ref = (red / rgb_gray_red_avg), (green / rgb_gray_green_avg), (blue / rgb_gray_blue_avg)
    
    # Calculate the SCI (Soil Color Index)
    sci = (rgb_red_ref - rgb_green_ref) / (rgb_red_ref + rgb_green_ref)
    
    # Plot the SCI image
    plt.figure()
    plt.imshow(sci, cmap='gray')
    plt.title('Soil Color Index (SCI)')
    plt.colorbar()
    # plt.savefig('../assets/vi/sci.png')
    
    # Histogram of SCI
    plt.figure()
    plt.hist(sci.ravel(), bins=256, range=(-1.0, 1.0), fc='k', ec='k')
    plt.title('Soil Color Index (SCI) Histogram')
    # plt.savefig('../assets/vi/sci_hist.png')
    
    # Use the NIR image to calculate the GNDVI (Green Normalized Difference Vegetation Index)
    
    # Extract a 5x5 region around the NIR center coordinates
    nir_gray = img[nir_y_center - 5:nir_y_center + 5, nir_x_center - 5:nir_x_center + 5, :]
    
    # Extract the red, green, and blue bands from the gray region
    nir_gray_red, nir_gray_green, nir_gray_blue = nir_gray[:, :, 2], nir_gray[:, :, 1], nir_gray[:, :, 0]
    nir_gray_red_avg, nir_gray_green_avg, nir_gray_blue_avg = np.mean(nir_gray_red), np.mean(nir_gray_green), np.mean(nir_gray_blue)
    
    # Convert the NIR image to red, green, blue bands
    nir_red, nir_green, nir_blue = nir[:, :, 2], nir[:, :, 1], nir[:, :, 0]
    nir_red_ref, nir_green_ref, nir_blue_ref = (nir_red / nir_gray_red_avg), (nir_green / nir_gray_green_avg), (nir_blue / nir_gray_blue_avg)
    
    # Calculate the GNDVI
    gndvi = (nir_red_ref - nir_green_ref) / (nir_red_ref + nir_green_ref)
    
    # Plot the GNDVI image
    plt.figure()
    plt.imshow(gndvi, cmap='gray')
    plt.title('Green Normalized Difference Vegetation Index (GNDVI)')
    plt.colorbar()
    # plt.savefig('../assets/vi/gndvi.png')
    
    # Histogram of GNDVI
    plt.figure()
    plt.hist(gndvi.ravel(), bins=256, range=(-1.0, 1.0), fc='k', ec='k')
    plt.title('Green Normalized Difference Vegetation Index (GNDVI) Histogram')
    # plt.savefig('../assets/vi/gndvi_hist.png')
    
    return sci, gndvi


If you want to see if it works, try it on this image!

In [ ]:
# Load the image
img = cv2.imread('../data/cam1/date_1-5-2024_10.0.11_1.png')

# Call the function
vi(img, 279, 243, 919, 209)

### **Delete Later**

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

def ndvi(nir, x_center, y_center):
    gray = nir[y_center-5:y_center+5, x_center-5:x_center+5, :]
    gray_red, gray_green, gray_blue = gray[:, :, 0], gray[:, :, 1], gray[:, :, 2]
    gray_red_avg = np.mean(gray_red)
    gray_green_avg = np.mean(gray_green)
    gray_blue_avg = np.mean(gray_blue)
    
    nir_red, nir_green, nir_blue = nir[:, :, 0], nir[:, :, 1], nir[:, :, 2]
    nir_red_ref = nir_red / gray_red_avg
    nir_green_ref = nir_green / gray_green_avg
    nir_blue_ref = nir_blue / gray_blue_avg
    
    ndvi = ((1.664 * nir_blue_ref) / (0.953 * nir_red_ref)) - 1

    # Plot the NDVI image
    plt.figure(figsize=(10, 7))
    plt.imshow(ndvi, cmap='gray')
    plt.title('NDVI Image')
    plt.axis('off')
    plt.colorbar()
    plt.show()
    
    return ndvi

def process_ndvi():
    for i in range(1, 9):
        nir_folder = f'../data/cam{i}_nir'
        ndvi_folder = f'../assets/cam{i}_ndvi'
        
        # Create the output folder if it doesn't exist
        if not os.path.exists(ndvi_folder):
            os.makedirs(ndvi_folder)
        
        for filename in os.listdir(nir_folder):
            if filename.endswith('.png'):
                nir_path = os.path.join(nir_folder, filename)
                nir_image = cv2.imread(nir_path)
                
                # Assuming the center is the middle of the image
                x_center = nir_image.shape[1] // 2
                y_center = nir_image.shape[0] // 2
                
                ndvi_image = ndvi(nir_image, x_center, y_center)
                
                # Save the NDVI image
                output_path = os.path.join(ndvi_folder, filename)
                cv2.imwrite(output_path, ndvi_image * 255)

# Run the function
process_ndvi()